# Inference of Gaussian model parameters

In [ ]:
import arviz as az
import chi
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import pints
import scipy.stats
import scipy.special
import seaborn as sns
sns.set_theme()

The data-generating distribution is
$$
    y \sim \mathcal{N}(\cdot | \mu = 1.3, \sigma ^2 = 4).
$$

We will assume that the variance of the distribution is known. Together with
the prior $\mu \sim \mathcal{N}(\cdot | \mu _0 = 0, \sigma _0^2 = 16)$, we
can use the conjugacy to compute the posterior distribution of $\mu $
$$
    p(\mu | \mathcal{D}) = \mathcal{N}(\cdot | \mu ', \sigma '^2 ),
$$
where
$$
    \mu' = \sigma '^2
    \left(\frac{\mu _0}{\sigma _0^2} + \frac{n\bar{y}}{\sigma ^2}\right),
    \quad
    \sigma '^2 = \frac{\sigma ^ 2\sigma _0^2}{\sigma ^2 + n\sigma _0^2}.
$$
Here, $n$ is the number of observations in the dataset
$\mathcal{D}=\{y_j^{\mathrm{obs}}\}$ and $\bar{y}$ denotes the empirical mean,
$\bar{y} = \sum _j y_j^{\mathrm{obs}} / n$.

## Synthesise data

In [ ]:
np.random.seed(12)
true_parameters = [1.3, 2]
data = np.random.normal(
    loc=true_parameters[0], scale=true_parameters[1], size=1000)

sns.displot(data, kind='kde', rug=True)
plt.show()

## Define log-likelihoods

1. Exact Gaussian log-likelihood
2. Naive Gaussian filter log-likelihood (stochastic)
3. Gaussian filter log-likelihood (deterministic)
4. Naive KDE filter log-likelihood (stochastic)
5. KDE filter log-likelihood (deterministic)

In [ ]:
class GaussianLogLikelihood(chi.LogLikelihood):
    def __init__(self, observations):
        self._observations = observations

    def __call__(self, parameters):
        mean = parameters[0]
        sigma = 2
        score = np.sum(
            scipy.stats.norm(loc=mean, scale=sigma).logpdf(self._observations))

        return score

    def n_parameters(self):
        return 1

    def get_parameter_names(self):
        return ['Mean']

    def get_id(self):
        return [None] * self.n_parameters()


class NaiveGaussianFilterLogLikelihood(chi.LogLikelihood):
    def __init__(self, observations, n_samples=1000):
        self._observations = observations
        self._n_samples = int(n_samples)

    def __call__(self, parameters):
        # Sample from population distribution
        mean = parameters[0]
        sigma = 2
        samples = np.random.normal(
            loc=mean, scale=sigma, size=self._n_samples)

        # Estimate mean and std from samples
        mean = np.mean(samples)
        sigma = np.std(samples, ddof=1)
        score = np.sum(
            scipy.stats.norm(loc=mean, scale=sigma).logpdf(self._observations))

        return score

    def n_parameters(self):
        return 1

    def get_parameter_names(self):
        return ['Mean']

    def get_id(self):
        return [None] * self.n_parameters()


class GaussianFilterLogLikelihood(chi.LogLikelihood):
    def __init__(self, observations, n_samples=1000):
        self._observations = observations
        self._n_samples = int(n_samples)
        self._normal = chi.GaussianErrorModel()

    def __call__(self, parameters):
        standard_samples = parameters[:self._n_samples]
        mean = parameters[-1]
        sigma = 2
        samples = mean + standard_samples * sigma

        # Estimate mean and std from samples
        mean = np.mean(samples)
        sigma = np.std(samples, ddof=1)
        score = np.sum(
            scipy.stats.norm(loc=mean, scale=sigma).logpdf(self._observations))

        return score

    def evaluateS1(self, parameters):
        standard_samples = np.array(parameters[:self._n_samples])
        mean = parameters[-1]
        std = 2
        samples = mean + standard_samples * std

        # Estimate mean and std from samples
        mu = np.mean(samples)
        sigma = np.std(samples, ddof=1)

        # Compute sensitivity of mean and sigma to input params
        dmu_dsamples = std / self._n_samples
        dmu_dmean = 1
        dsigma_dsamples = \
            (samples - mu) * std / sigma / (self._n_samples - 1)
        dsigma_dmean = 0

        # Propagate sensitivies through likelihood
        score, sens = self._normal.compute_sensitivities(
            parameters=[sigma],
            model_output=np.array([mu] * len(self._observations)),
            model_sensitivities=np.broadcast_to(np.hstack([
                dmu_dsamples, dmu_dmean]),
                shape=(len(self._observations), 2)),
            observations=self._observations)

        # Collect sensitivities
        # dp/dsamples = dp/dmu * dmu/dsample + dp/dsigma * dsigma/dsample
        # dp/dmean = dp/dmu * dmu/dmean + dp/dsigma * dsigma/dmean
        # dp/dstd = dp/dmu * dmu/dstd + dp/dsigma * dsigma/dstd
        sensitivities = np.empty(shape=self.n_parameters())
        sensitivities[:self._n_samples] = sens[0] + sens[2] * dsigma_dsamples
        sensitivities[self._n_samples] = sens[1] + sens[2] * dsigma_dmean

        return score, sensitivities

    def n_parameters(self):
        return self._n_samples + 1

    def get_parameter_names(self):
        return [
            'Sample' for _ in range(self._n_samples)] + ['Mean']

    def get_id(self):
        return ['ID %d' % _id for _id in range(self._n_samples)] + [None]


class NaiveKDEFilterLogLikelihood(chi.LogLikelihood):
    def __init__(self, observations, n_samples=1000, kernel_scale=None):
        self._observations = observations
        self._n_obs = len(observations)
        self._n_samples = int(n_samples)
        self._normal = chi.GaussianErrorModel()

        # Set kernel scale
        # (If None or non-positive use Scott's rule of thumb (like scipy))
        if (not kernel_scale) or (kernel_scale <=0):
            kernel_scale = self._n_samples ** (-0.2)
        self._kernel_scale = kernel_scale

    def __call__(self, parameters):
        mean = parameters[0]
        sigma = 2
        samples = np.random.normal(
            loc=mean, scale=sigma, size=self._n_samples)

        # Estimate log-likelihood as the mean across kdes
        score = np.sum(scipy.special.logsumexp(
            - np.log(2 * np.pi) / 2 - np.log(self._kernel_scale) - \
            (samples[np.newaxis, :] - self._observations[:, np.newaxis])**2 \
            / self._kernel_scale**2 / 2, axis=1) - np.log(self._n_samples))

        return score

    def n_parameters(self):
        return 1

    def get_parameter_names(self):
        return ['Mean']

    def get_id(self):
        return [None]


class KDEFilterLogLikelihood(chi.LogLikelihood):
    def __init__(self, observations, n_samples=1000, kernel_scale=None):
        self._observations = observations
        self._n_obs = len(observations)
        self._n_samples = int(n_samples)
        self._normal = chi.GaussianErrorModel()

        # Set kernel scale
        # (If None or non-positive use Scott's rule of thumb (like scipy))
        if (not kernel_scale) or (kernel_scale <=0):
            kernel_scale = self._n_samples ** (-0.2)
        self._kernel_scale = kernel_scale

    def __call__(self, parameters):
        standard_samples = parameters[:self._n_samples]
        mean = parameters[-1]
        sigma = 2
        samples = mean + standard_samples * sigma

        # Estimate log-likelihood as the mean across kdes
        score = np.sum(scipy.special.logsumexp(
            - np.log(2 * np.pi) / 2 - np.log(self._kernel_scale) - \
            (samples[np.newaxis, :] - self._observations[:, np.newaxis])**2 \
            / self._kernel_scale**2 / 2, axis=1) - np.log(self._n_samples))

        return score

    def evaluateS1(self, parameters):
        standard_samples = np.array(parameters[:self._n_samples])
        mean = parameters[-1]
        std = 2
        samples = mean + standard_samples * std

        # Estimate log-likelihood as the mean across kdes
        scores = \
            - np.log(2 * np.pi) / 2 - np.log(self._kernel_scale) - \
            (samples[np.newaxis, :] - self._observations[:, np.newaxis])**2 \
            / self._kernel_scale**2 / 2
        score = np.sum(
            scipy.special.logsumexp(scores, axis=1) - np.log(self._n_samples))

        # Collect sensitivities
        # p = log mean exp scores
        # dp/dsamples = exp(score) / sum(exp(scores)) * dscore / dsamples
        # dp/dmean = exp(score) / sum(exp(scores)) * dscore / dmu
        softmax = \
            np.exp(scores - np.max(scores, axis=1)[:, np.newaxis]) / np.sum(
            np.exp(scores - np.max(scores, axis=1)[:, np.newaxis]),
            axis=1)[:, np.newaxis]

        sensitivities = np.empty(shape=self.n_parameters())
        sensitivities[:self._n_samples] = \
            - std / self._kernel_scale**2 * np.sum(
                (samples[np.newaxis, :] - self._observations[:, np.newaxis]) *
                softmax,
                axis=0)
        sensitivities[self._n_samples] = \
            - 1 / self._kernel_scale**2 * np.sum(
                (samples[np.newaxis, :] - self._observations[:, np.newaxis]) *
                softmax)

        return score, sensitivities

    def n_parameters(self):
        return self._n_samples + 1

    def get_parameter_names(self):
        return [
            'Sample' for _ in range(self._n_samples)] + ['Mean']

    def get_id(self):
        return ['ID %d' % _id for _id in range(self._n_samples)] + [None]

## Compare log-likelihood for fixed parameters

In [ ]:
n_repeats = 1000

# Compute exact log-likelihood
scores_exact = np.empty(n_repeats)
log_likelihood = GaussianLogLikelihood(observations=data)
for idr in range(n_repeats):
    scores_exact[idr] = log_likelihood(true_parameters)

# Compute naive Gaussian KDE log-likelihood
scores_naive = np.empty(n_repeats)
log_likelihood = NaiveGaussianFilterLogLikelihood(
    observations=data, n_samples=1000)
for idr in range(n_repeats):
    scores_naive[idr] = log_likelihood(true_parameters)

# Compute Gaussian KDE log-likelihood
scores = np.empty(n_repeats)
n_samples = 1000
parameters = np.hstack([
    np.random.normal(size=n_samples), true_parameters])
log_likelihood = GaussianFilterLogLikelihood(
    observations=data, n_samples=n_samples)
for idr in range(n_repeats):
    scores[idr] = log_likelihood(parameters)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(n_repeats),
    y=scores_exact,
    name='Exact'
))
fig.add_trace(go.Scatter(
    x=np.arange(n_repeats),
    y=scores_naive,
    name='Naive Gaussian Filter (1000 samples)'
))
fig.add_trace(go.Scatter(
    x=np.arange(n_repeats),
    y=scores,
    name='Gaussian Filter (1000 samples)'
))
fig.update_layout(
    xaxis_title='Repeat',
    yaxis_title='Log-likelihood'
)
fig.show()

## Inference

### 1. Exact likelihood

In [ ]:
log_likelihood = GaussianLogLikelihood(observations=data)
log_prior = pints.GaussianLogPrior(mean=0, sd=4)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
n_iterations = 10000
exact_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 1000
thinning = 10
az.plot_trace(
    exact_posterior_samples.sel(draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

### 2. Naive Gaussian filter likelihood

10 samples per iteration

In [ ]:
log_likelihood = NaiveGaussianFilterLogLikelihood(
    observations=data, n_samples=10)
log_prior = pints.GaussianLogPrior(mean=0, sd=4)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_sampler(sampler=pints.MetropolisRandomWalkMCMC)
controller.set_n_runs(1)
controller._initial_params[0, 0] = 1.2
n_iterations = 50000
naive_10_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 1000
thinning = 10
az.plot_trace(
    naive_10_posterior_samples.sel(draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

100 samples per iteration

In [ ]:
log_likelihood = NaiveGaussianFilterLogLikelihood(
    observations=data, n_samples=100)
log_prior = pints.GaussianLogPrior(mean=0, sd=4)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_sampler(sampler=pints.MetropolisRandomWalkMCMC)
controller.set_n_runs(1)
controller._initial_params[0, 0] = 1.2
n_iterations = 50000
naive_100_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 1000
thinning = 10
az.plot_trace(
    naive_100_posterior_samples.sel(draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

1000 samples per iteration

In [ ]:
log_likelihood = NaiveGaussianFilterLogLikelihood(
    observations=data, n_samples=1000)
log_prior = pints.GaussianLogPrior(mean=0, sd=4)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_sampler(sampler=pints.MetropolisRandomWalkMCMC)
controller.set_n_runs(1)
controller._initial_params[0, 0] = 1.2
n_iterations = 50000
naive_1000_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 1000
thinning = 10
az.plot_trace(
    naive_1000_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

### 3. Gaussian filter likelihood

10 samples

In [ ]:
n_samples = 10
log_likelihood = GaussianFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.ComposedLogPrior(*[
    pints.GaussianLogPrior(mean=0, sd=1)] * n_samples + [
    pints.GaussianLogPrior(mean=0, sd=4)]
)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.NoUTurnMCMC)
n_iterations = 2000
gaussian_filter_10_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 1000
thinning = 1
az.plot_trace(
    gaussian_filter_10_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

100 samples

In [ ]:
n_samples = 100
log_likelihood = GaussianFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.ComposedLogPrior(*[
    pints.GaussianLogPrior(mean=0, sd=1)] * n_samples + [
    pints.GaussianLogPrior(mean=0, sd=4)]
)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.NoUTurnMCMC)
n_iterations = 1500
gaussian_filter_100_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 500
thinning = 1
az.plot_trace(
    gaussian_filter_100_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

1000 samples

In [ ]:
n_samples = 1000
log_likelihood = GaussianFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.ComposedLogPrior(*[
    pints.GaussianLogPrior(mean=0, sd=1)] * n_samples + [
    pints.GaussianLogPrior(mean=0, sd=4)]
)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.NoUTurnMCMC)
n_iterations = 1500
gaussian_filter_1000_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 500
thinning = 1
az.plot_trace(
    gaussian_filter_1000_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

10000 samples

In [ ]:
n_samples = 10000
log_likelihood = GaussianFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.ComposedLogPrior(*[
    pints.GaussianLogPrior(mean=0, sd=1)] * n_samples + [
    pints.GaussianLogPrior(mean=0, sd=4)]
)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.NoUTurnMCMC)
n_iterations = 1500
gaussian_filter_10000_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 500
thinning = 1
az.plot_trace(
    gaussian_filter_10000_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

In [ ]:
gaussian_filter_10000_posterior_samples.to_netcdf(
    'derived_data/posteriors/gaussian_model_gaussian_filter_10000_samples.nc')

## 4. Naive KDE inference

10 samples

In [ ]:
n_samples = 10
log_likelihood = NaiveKDEFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.GaussianLogPrior(mean=0, sd=4)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.MetropolisRandomWalkMCMC)
n_iterations = 200000
naive_kde_filter_10_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 20000
thinning = 1000
az.plot_trace(
    naive_kde_filter_10_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

100 samples

In [ ]:
n_samples = 100
log_likelihood = NaiveKDEFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.GaussianLogPrior(mean=0, sd=4)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.MetropolisRandomWalkMCMC)
n_iterations = 200000
naive_kde_filter_100_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 20000
thinning = 1000
az.plot_trace(
    naive_kde_filter_100_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

1000 samples

In [ ]:
n_samples = 1000
log_likelihood = NaiveKDEFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.GaussianLogPrior(mean=0, sd=4)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.MetropolisRandomWalkMCMC)
n_iterations = 50000
naive_kde_filter_1000_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 1000
thinning = 10
az.plot_trace(
    naive_kde_filter_1000_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

### 5. KDE filter inference

10 samples

In [ ]:
n_samples = 10
log_likelihood = KDEFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.ComposedLogPrior(*[
    pints.GaussianLogPrior(mean=0, sd=1)] * n_samples + [
    pints.GaussianLogPrior(mean=0, sd=4)]
)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.NoUTurnMCMC)
n_iterations = 1500
kde_filter_10_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=False)

warmup = 500
thinning = 1
az.plot_trace(
    kde_filter_10_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

100 samples

In [ ]:
n_samples = 100
log_likelihood = KDEFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.ComposedLogPrior(*[
    pints.GaussianLogPrior(mean=0, sd=1)] * n_samples + [
    pints.GaussianLogPrior(mean=0, sd=4)]
)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.NoUTurnMCMC)
n_iterations = 1500
kde_filter_100_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=True)

warmup = 500
thinning = 1
az.plot_trace(
    kde_filter_100_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

1000 samples

In [ ]:
n_samples = 1000
log_likelihood = KDEFilterLogLikelihood(
    observations=data, n_samples=n_samples)
log_prior = pints.ComposedLogPrior(*[
    pints.GaussianLogPrior(mean=0, sd=1)] * n_samples + [
    pints.GaussianLogPrior(mean=0, sd=4)]
)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

controller = chi.SamplingController(log_posterior)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.NoUTurnMCMC)
n_iterations = 1500
kde_filter_1000_posterior_samples = controller.run(
    n_iterations=n_iterations, log_to_screen=True)

warmup = 500
thinning = 1
az.plot_trace(
    kde_filter_1000_posterior_samples.sel(
        draw=slice(warmup, n_iterations, thinning)))
plt.tight_layout()

In [ ]:
kde_filter_1000_posterior_samples.to_netcdf(
    'derived_data/posteriors/gaussian_model_kde_filter_1000_samples.nc')

## Comparison to analytic solution

In [ ]:
# Get true distribution
n = len(data)
mu_0 = 0
var_0 = 16
true_mu, true_std = true_parameters
true_var = true_std ** 2
posterior_var = var_0 * true_var / (true_var + n * var_0)
posterior_mean = posterior_var * (
    mu_0 / var_0 + n * np.mean(data) / true_var)
mus = np.linspace(1, 1.6, num=200)
true_pdf = scipy.stats.norm(posterior_mean, np.sqrt(posterior_var)).pdf(mus)

# Create figure
fig, axes = plt.subplots(
    1, 3, figsize=(10, 5), sharey='row', sharex='row')
plt.subplots_adjust(wspace=0.1, hspace=0.3)
axes[0].set_xlim([posterior_mean - 1, posterior_mean + 1])
axes[0].set_xlabel('Mu')
axes[1].set_xlabel('Mu')
axes[2].set_xlabel('Mu')
colors = sns.color_palette()

# Plot exact likelihood inference
warmup = 1000
thinning = 10
mu_samples = exact_posterior_samples.sel(
    draw=slice(warmup, n_iterations, thinning))['Mean'].values[0]
sns.kdeplot(
    x=mu_samples,
    fill=True,
    common_norm=False, alpha=.5, linewidth=1, ax=axes[0],
    palette=colors[0], legend=False)

# Plot Gaussian filter posteriors
warmup = 1000
thinning = 1
mu_samples = gaussian_filter_10_posterior_samples.sel(
    draw=slice(warmup, n_iterations, thinning))['Mean'].values[0]
sns.kdeplot(
    x=mu_samples,
    fill=True,
    common_norm=False, alpha=.5, linewidth=1, ax=axes[1],
    palette=colors[0], legend=False)

warmup = 500
thinning = 1
mu_samples = gaussian_filter_100_posterior_samples.sel(
    draw=slice(warmup, n_iterations, thinning))['Mean'].values[0]
sns.kdeplot(
    x=mu_samples,
    fill=True,
    common_norm=False, alpha=.5, linewidth=1, ax=axes[1],
    palette=colors[1], legend=False)

warmup = 500
thinning = 1
mu_samples = gaussian_filter_1000_posterior_samples.sel(
    draw=slice(warmup, n_iterations, thinning))['Mean'].values[0]
sns.kdeplot(
    x=mu_samples,
    fill=True,
    common_norm=False, alpha=.5, linewidth=1, ax=axes[1],
    palette=colors[2], legend=False)

warmup = 500
thinning = 1
mu_samples = gaussian_filter_10000_posterior_samples.sel(
    draw=slice(warmup, n_iterations, thinning))['Mean'].values[0]
sns.kdeplot(
    x=mu_samples,
    fill=True,
    common_norm=False, alpha=.5, linewidth=1, ax=axes[1],
    palette=colors[3], legend=False)

# Plot KDE filter posteriors
warmup = 500
thinning = 1
mu_samples = kde_filter_10_posterior_samples.sel(
    draw=slice(warmup, n_iterations, thinning))['Mean'].values[0]
sns.kdeplot(
    x=mu_samples,
    fill=True,
    common_norm=False, alpha=.5, linewidth=1, ax=axes[2],
    palette=colors[0], legend=False)

warmup = 500
thinning = 1
mu_samples = kde_filter_100_posterior_samples.sel(
    draw=slice(warmup, n_iterations, thinning))['Mean'].values[0]
sns.kdeplot(
    x=mu_samples,
    fill=True,
    common_norm=False, alpha=.5, linewidth=1, ax=axes[2],
    palette=colors[1], legend=False)

warmup = 500
thinning = 1
mu_samples = kde_filter_1000_posterior_samples.sel(
    draw=slice(warmup, n_iterations, thinning))['Mean'].values[0]
sns.kdeplot(
    x=mu_samples,
    fill=True,
    common_norm=False, alpha=.5, linewidth=1, ax=axes[2],
    palette=colors[1], legend=False)

# Overlay exact posterior
axes[0].plot(
    mus, true_pdf, color='black', linestyle='--', label='Analytic posterior')
axes[1].plot(
    mus, true_pdf, color='black', linestyle='--')
axes[2].plot(
    mus, true_pdf, color='black', linestyle='--')

fig.legend()
plt.show()